In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os
np.random.seed(0) #Code B

### create G.19


In [19]:
a=1
b=2
c=3
d=4
df = pd.DataFrame({'X':[a,b,c,b,a,a,a,b,b,b], 
                   'Y':[b,c,d,a,c,d,a,d,a,b]})
df['Fraudster--X--Y'] = True
df['Fraud_Transaction--X--Y'] = df['X']!=df['Y']

In [20]:
create_facts_and_examples(df_= df, target='Fraudster--X--Y', 
                          predicates=['Fraud_Transaction--X--Y'], 
                          output_dir="examples/G.19")

Fraudster--X--Y 1 2 flag= True
Fraudster--X--Y 2 3 flag= True
Fraudster--X--Y 3 4 flag= True
Fraudster--X--Y 2 1 flag= True
Fraudster--X--Y 1 3 flag= True
Fraudster--X--Y 1 4 flag= True
Fraudster--X--Y 2 4 flag= True
Fraudster--X--Y 2 1 flag= True
Fraud_Transaction--X--Y
Fraud_Transaction--X--Y 1 2 flag= True
Fraud_Transaction--X--Y 2 3 flag= True
Fraud_Transaction--X--Y 3 4 flag= True
Fraud_Transaction--X--Y 2 1 flag= True
Fraud_Transaction--X--Y 1 3 flag= True
Fraud_Transaction--X--Y 1 4 flag= True
Fraud_Transaction--X--Y 2 4 flag= True
Fraud_Transaction--X--Y 2 1 flag= True


In [21]:
df

,X,Y,Fraudster--X--Y,Fraud_Transaction--X--Y
0,1,2,True,True
1,2,3,True,True
2,3,4,True,True
3,2,1,True,True
4,1,3,True,True
5,1,4,True,True
6,1,1,True,False
7,2,4,True,True
8,2,1,True,True
9,2,2,True,False


### Transitivity

In [5]:
Fraud = np.full(2, False)
Fraud

array([False, False])

In [6]:
N = 10

customer = ['c'+str(i) for i in range(100)]

Fraud = np.full(int(N/2)+1, int(0))
Fraud[-2:]=int(1)
df = pd.DataFrame({'orig': customer[0:N:2]+[customer[N-1]], 
                   'destination':customer[1:N:2]+[customer[N]],\
                   "Fraud":Fraud})


In [7]:
df

,orig,destination,Fraud
0,c0,c1,0
1,c2,c3,0
2,c4,c5,0
3,c6,c7,0
4,c8,c9,1
5,c9,c10,1


In [8]:
df.drop(df[df['orig']==df['destination']].index, inplace=True)

In [9]:
fraudsters = df[df['Fraud']==True]

In [10]:
fraudsters[['orig','destination']]

,orig,destination
4,c8,c9
5,c9,c10


In [11]:
index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), how='inner', left_on='orig', right_on='destination')

In [12]:
index_from_merge

,index,orig,destination_x,Fraud,destination_y
0,5,c9,c10,1,c9


In [13]:
df['Fraud_chain--orig--destination']=0
df.loc[index_from_merge['index'],'Fraud']=1
df.loc[index_from_merge['index'],'Fraud_chain--orig--destination']=1
df['From--orig']=1
df['To--destination']=1

In [14]:
df.loc[index_from_merge['index'],:]

,orig,destination,Fraud,Fraud_chain--orig--destination,From--orig,To--destination
5,c9,c10,1,1,1,1


In [15]:
df

,orig,destination,Fraud,Fraud_chain--orig--destination,From--orig,To--destination
0,c0,c1,0,0,1,1
1,c2,c3,0,0,1,1
2,c4,c5,0,0,1,1
3,c6,c7,0,0,1,1
4,c8,c9,1,0,1,1
5,c9,c10,1,1,1,1


### creating fact, positive and negative examples

In [16]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [17]:
df['Fraud--orig--destination']=df['Fraud']
df['Transfer--orig--destination']=1
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination', 'Transfer--orig--destination'], 
                          output_dir="examples/debug-simple-transitivity")

Fraud_chain--orig--destination 9 10 flag= True
Fraud_chain--orig--destination 0 1 flag= False
Fraud_chain--orig--destination 2 3 flag= False
Fraud_chain--orig--destination 4 5 flag= False
Fraud_chain--orig--destination 6 7 flag= False
Fraud_chain--orig--destination 8 9 flag= False
Fraud--orig--destination
Fraud--orig--destination 8 9 flag= True
Fraud--orig--destination 9 10 flag= True
Transfer--orig--destination
Transfer--orig--destination 0 1 flag= True
Transfer--orig--destination 2 3 flag= True
Transfer--orig--destination 4 5 flag= True
Transfer--orig--destination 6 7 flag= True
Transfer--orig--destination 8 9 flag= True
Transfer--orig--destination 9 10 flag= True


In [18]:
df

,orig,destination,Fraud,Fraud_chain--orig--destination,From--orig,To--destination,Fraud--orig--destination,Transfer--orig--destination
0,0,1,0,0,1,1,0,1
1,2,3,0,0,1,1,0,1
2,4,5,0,0,1,1,0,1
3,6,7,0,0,1,1,0,1
4,8,9,1,0,1,1,1,1
5,9,10,1,1,1,1,1,1
